# Modelos para probabilidad de respuesta

Importamos librerías:

In [269]:
# Procesamiento y gráficos
import os
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
# Modelos
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, confusion_matrix
from ISLP.models import (ModelSpec,
                         Stepwise,
                         sklearn_selected)

# Selección de variables
from ISLP.models import (Stepwise,sklearn_selected)  # Para selección de variables (ISLP)
from sklearn.feature_selection import SequentialFeatureSelector # Selección de variables (CV)

Seteamos el directorio de trabajo:

In [270]:
os.chdir('C:/Users/celin/OneDrive/Documentos/CELINE/FACULTAD/Maestría en Estadística Aplicada/4to semestre/Tesis/Nonresponse-bias-eph')

Cargamos la base de datos individual (filtrada por el jefe de hogar, para tomar un registro por hogar):

In [271]:
# Cargamos las bases de datos
individual_NEA = pd.read_csv("Bases/individual_NEA_paramodelos.txt", sep=";", low_memory=False)
individual_NEA.head()

,ANO4,TRIMESTRE,CODUSU,NRO_HOGAR,periodo,AGLOMERADO,REGION_DESC,REGION_COD,CH04,CH06,...,II4_2,II4_3,II5,II5_1,II6,II6_1,II7,II8,II9,viviendasxarea
0,2017,2,TQRMNOPPQHJMKRCDEFOCD00472238,1,2017.25,8,Noreste,41,1,62,...,2.0,2.0,2.0,0.0,2.0,0.0,1.0,2.0,1.0,7
1,2017,2,TQRMNOPPQHKMLMCDEGLDF00590955,1,2017.25,15,Noreste,41,2,48,...,1.0,2.0,2.0,0.0,2.0,0.0,3.0,2.0,1.0,3
2,2017,2,TQRMNOPPQHKMLQCDEFOCD00472357,1,2017.25,8,Noreste,41,1,39,...,1.0,1.0,2.0,0.0,2.0,0.0,1.0,2.0,1.0,7
3,2017,2,TQRMNOPPQHKOKOCDEGIBJ00509386,1,2017.25,12,Noreste,41,1,34,...,2.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,9
4,2017,2,TQRMNOPPRHJMKRCDEFOCD00472239,1,2017.25,8,Noreste,41,1,33,...,1.0,2.0,2.0,0.0,2.0,0.0,1.0,2.0,1.0,7


In [273]:
# Dimensión de la base
print("Dimensión de la base total: ", individual_NEA.shape)

Dimensión de la base total:  (36172, 259)


Filtramos por la última entrevista registrada de cada hogar, para no duplicar:

In [274]:
# Filtramos por última entrevista
ult_entrevista = individual_NEA.groupby(['CODUSU','NRO_HOGAR'])['n_entrevista'].idxmax()
individual_NEA = individual_NEA.loc[ult_entrevista].reset_index(drop=True)
# Cantidad de registros
print("Dimensión de la base filtrada: ", individual_NEA.shape)

Dimensión de la base filtrada:  (15814, 259)


In [275]:
# Variables para el modelo
variables = [
    'completo',  # Variable de respuesta
    # Variables de identificación del hogar
    'AGLO_DESC', 'periodo', 'PONDERA_repr', 'viviendasxarea',
    # Ingreso del hogar
    'IPCF_d', 'otros_ing_nolab', 'RDECOCUR', 'ADECOCUR',
    # Datos del jefe de hogar
    'CH06', 'CH06_2', 'IX_TOT', 'informal', 'NIVEL_ED', 'mujer', 'casadounido', 'ESTADO', 'CAT_INAC', 'CAT_OCUP',
    # Características de la vivienda
    'IV1', 'IV2', 'IV3', 'IV4', 'IV5', 'IV6', 'IV7', 'IV8', 'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3',
    # Características habitacionales del hogar
    'II1', 'II2', 'II3', 'II3_1', 'II4_1', 'II4_2', 'II4_3', 'II5', 'II6', 'II6_1', 'II7', 'II8', 'II9',
    # Ocupación del jefe de hogar
    'CALIFICACION', 'JERARQUIA', 'caes_seccion_cod', 'PP04B1',
    # Características de composición del hogar
    'cantidad_varones', 'cantidad_mujeres', 'cantidad_ocupados','cantidad_desocupados', 'cantidad_informales', 'edad_promedio_hogar'
]

# Seleccionamos estos campos en la base de datos
individual_NEA = individual_NEA[variables]

# Estratos para división
individual_NEA['estrato'] = (
    individual_NEA['completo'].astype(str) + '_' +
    individual_NEA['periodo'].astype(str)
)

Convertimos las variables tipo string a categóricas y luego a dummies:

In [276]:
# Convertimos en categóricas las variables tipo object
for col in individual_NEA.select_dtypes(include='object').columns:
    individual_NEA[col] = individual_NEA[col].astype('category')
# Convertimos en dummies las variables categóricas
categoricas = ['NIVEL_ED','CALIFICACION','JERARQUIA','caes_seccion_cod','PP04B1']
individual_NEA = pd.get_dummies(individual_NEA, columns=categoricas, drop_first=True, dtype=int)
individual_NEA = individual_NEA.drop(columns=['JERARQUIA_No ocupado','caes_seccion_cod_No ocupado','PP04B1'])

Borramos un par de registros que están mal (controlar esto):

In [277]:
print('Dimensión de la base (antes): ', individual_NEA.shape)
individual_NEA = individual_NEA[
    ~(
        (individual_NEA['completo'] == 1) &
        (individual_NEA['periodo'] == 2019.75) &
        (individual_NEA['AGLO_DESC'] == 'Gran Resistencia')
    )
]

individual_NEA = individual_NEA[
    ~(
        (individual_NEA['completo'] == 1) &
        (individual_NEA['periodo'] == 2020.75) &
        (individual_NEA['AGLO_DESC'] == 'Gran Resistencia')
    )
]
print('Dimensión de la base (después): ', individual_NEA.shape)

Dimensión de la base (antes):  (15814, 57)
Dimensión de la base (después):  (15812, 57)


In [278]:
print('Dimensión de la base (antes): ', individual_NEA.shape)
individual_NEA = individual_NEA.dropna()
print('Dimensión de la base (después): ', individual_NEA.shape)

Dimensión de la base (antes):  (15812, 57)
Dimensión de la base (después):  (15810, 57)


Definimos los vectores y, x y estrato para la división en entrenamiento y test:

In [279]:
# respuesta de los modelos
y_R = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Gran Resistencia']['completo'])
y_C = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Corrientes']['completo'])
y_F = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Formosa']['completo'])
y_P = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Posadas']['completo'])
# Matriz del modelo
X_R = individual_NEA[individual_NEA['AGLO_DESC']=='Gran Resistencia'].drop(columns=['completo','estrato','AGLO_DESC'])
X_C = individual_NEA[individual_NEA['AGLO_DESC']=='Corrientes'].drop(columns=['completo','estrato','AGLO_DESC'])
X_F = individual_NEA[individual_NEA['AGLO_DESC']=='Formosa'].drop(columns=['completo','estrato','AGLO_DESC'])
X_P = individual_NEA[individual_NEA['AGLO_DESC']=='Posadas'].drop(columns=['completo','estrato','AGLO_DESC'])
# Variables para estratificar
estrato_R = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Gran Resistencia']['estrato'])
estrato_C = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Corrientes']['estrato'])
estrato_F = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Formosa']['estrato'])
estrato_P = np.array(individual_NEA[individual_NEA['AGLO_DESC']=='Posadas']['estrato'])

División en entrenamiento y test, dividimos internamente:

In [280]:
X_R_train, X_R_test, y_R_train, y_R_test = train_test_split(X_R, y_R, test_size=0.2, stratify=estrato_R) # Gran Resistencia
X_C_train, X_C_test, y_C_train, y_C_test = train_test_split(X_C, y_C, test_size=0.2, stratify=estrato_C) # Corrientes
X_F_train, X_F_test, y_F_train, y_F_test = train_test_split(X_F, y_F, test_size=0.2, stratify=estrato_F) # Formosa
X_P_train, X_P_test, y_P_train, y_P_test = train_test_split(X_P, y_P, test_size=0.2, stratify=estrato_P) # Posadas

In [281]:
print(f'Proporciones en datos originales \n {pd.Series(y).value_counts(normalize=True)} \n')
print(f'Proporciones en train \n {pd.Series(y_train).value_counts(normalize=True)} \n')
print(f'Proporciones en test \n {pd.Series(y_test).value_counts(normalize=True)}')

Proporciones en datos originales 
 1    0.589357
0    0.410643
Name: proportion, dtype: float64 

Proporciones en train 
 0    0.508197
1    0.491803
Name: proportion, dtype: float64 

Proporciones en test 
 0    0.736364
1    0.263636
Name: proportion, dtype: float64


Revisar casos con NAs en las variables regresoras:

## Modelo de regresión logístico

 Realizamos selección de variables usando stepwise forward selection para el modelo logístico (escalamos variables previamente):

In [255]:
def negAIC(estimator, X, Y):
    "Negative AIC"
    n, p = X.shape
    Yhat = estimator.predict(X)
    MSE = np.mean((Y - Yhat)**2)
    return n + n * np.log(MSE) + 2 * (p + 1)

def drop_singular_cols(X):
    # Rangos
    rank = np.linalg.matrix_rank(X)
    while rank < X.shape[1]:
        # Calcular correlación perfecta
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [col for col in upper.columns if any(upper[col] == 1)]
        if not to_drop:
            break
        X = X.drop(columns=[to_drop[0]])
    return X.drop(columns='const', errors='ignore')

In [283]:
# Diccionario con los datasets
datasets = {
    "Resistencia": (X_R_train, y_R_train),
    "Corrientes": (X_C_train, y_C_train),
    "Formosa": (X_F_train, y_F_train),
    "Posadas": (X_P_train, y_P_train)
}

# Matrices de diseño
print('Matrices de diseño. Cálculo.')
designs = {}
for aglo, (X_train, y_train) in datasets.items():
    design = ModelSpec(X_train.columns).fit(X_train)
    X_train = design.transform(X_train)
    designs[aglo] = design
    datasets[aglo] = (X_train, y_train)
    print(f"✅ Matriz de diseño para {aglo}")

Matrices de diseño. Cálculo.


ValueError: 'B' is not in list

In [257]:
for aglo, (X, y) in datasets.items():
    # Agregar constante para intercepto (requerido por statsmodels)    
    # Calcular rango
    rank = np.linalg.matrix_rank(X_const)
    n_cols = X.shape[1]
    
    if rank < n_cols:
        print(f"{aglo}: Matriz SINGULAR (rango={rank}, columnas={n_cols})")
    else:
        print(f"{aglo}: Matriz OK (rango={rank}, columnas={n_cols})")

def columnas_singulares(X):
    X_const = sm.add_constant(X, has_constant='add')
    singular_cols = []
    while np.linalg.matrix_rank(X_const) < X_const.shape[1]:
        # Matriz de correlación absoluta
        corr_matrix = X_const.corr().abs()
        # Upper triangle
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        # Columnas con correlación perfecta
        to_drop = [col for col in upper.columns if any(upper[col] == 1)]
        if not to_drop:
            break
        singular_cols.append(to_drop[0])
        X_const = X_const.drop(columns=[to_drop[0]])
    return singular_cols

# Ejemplo para un dataset del diccionario
for aglo, (X, y) in datasets.items():
    cols_sing = columnas_singulares(X)
    if cols_sing:
        print(f"{aglo}: Columnas causando singularidad -> {cols_sing}")
    else:
        print(f"{aglo}: Ninguna columna singular")

Resistencia: Matriz SINGULAR (rango=83, columnas=85)
Corrientes: Matriz SINGULAR (rango=83, columnas=85)
Formosa: Matriz SINGULAR (rango=83, columnas=85)
Posadas: Matriz SINGULAR (rango=83, columnas=85)
Resistencia: Ninguna columna singular
Corrientes: Ninguna columna singular
Formosa: Ninguna columna singular
Posadas: Ninguna columna singular


In [284]:
X_train

,periodo,PONDERA_repr,viviendasxarea,IPCF_d,otros_ing_nolab,RDECOCUR,ADECOCUR,CH06,CH06_2,IX_TOT,...,CALIFICACION,JERARQUIA,caes_seccion_cod,PP04B1,cantidad_varones,cantidad_mujeres,cantidad_ocupados,cantidad_desocupados,cantidad_informales,edad_promedio_hogar
8400,2017.75,256.000000,8,1425.142961,0.0,0,0,26,676,4,...,No ocupado,No ocupado,No ocupado,No ocupado,2,2,1,0,0,14.25
9731,2022.75,359.750000,11,2007.569653,0.0,0,0,57,3249,2,...,No ocupado,No ocupado,No ocupado,No ocupado,1,1,0,0,0,67.50
12001,2023.50,328.583333,10,4212.955922,0.0,7,9,39,1521,2,...,Operativos,Cuenta propia,F,No presta,1,1,2,0,2,38.00
15452,2022.50,348.250000,4,1050.394754,0.0,0,0,29,841,3,...,No ocupado,No ocupado,No ocupado,No ocupado,1,2,0,0,0,11.00
11989,2021.25,519.666667,6,344.915131,0.0,1,1,43,1849,5,...,No calificados,Trabajadores asalariados,T,Presta servicio,0,5,1,0,0,24.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11029,2020.75,407.000000,6,14868.208766,0.0,9,9,46,2116,1,...,Operativos,Trabajadores asalariados,O,No presta,1,0,1,0,0,46.00
9389,2018.25,302.076923,12,3381.270841,7200.0,0,0,62,3844,2,...,No ocupado,No ocupado,No ocupado,No ocupado,1,1,1,0,1,62.00
5639,2023.50,295.071429,12,5687.490495,150000.0,0,0,78,6084,2,...,No ocupado,No ocupado,No ocupado,No ocupado,0,2,0,0,0,73.50
13877,2023.25,251.000000,1,8193.270131,0.0,8,9,52,2704,2,...,Operativos,Trabajadores asalariados,O,No presta,1,1,2,0,0,51.50


In [266]:
# Crear el modelo Logit

# Extraemos X e y del diccionario
X_train, y_train = datasets['Corrientes']
modelo_logit = sm.Logit(y_train, X_train)

# Ajustar el modelo
resultado = modelo_logit.fit()

# Ver resumen del modelo
print(resultado.summary())

         Current function value: inf
         Iterations: 35


C:\Users\celin\anaconda3\envs\islp\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\celin\anaconda3\envs\islp\Lib\site-packages\statsmodels\discrete\discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


LinAlgError: Singular matrix

In [259]:
# Diccionario para modelos ajustados
modelos = {}

# Stepwise
for aglo, (X_train, y_train) in datasets.items():
    
    # Establecemos estrategia
    print(f'Trabajando con {aglo}...')    
    design = designs[aglo]
    strategy = Stepwise.first_peak(
        design, 
        direction='forward',
        max_terms=len(design.terms)
    )

    # Corremos stepwise
    #modelo_LR = LogisticRegression(class_weight='balanced')
    modelo = sklearn_selected(
        sm.Logit,
        strategy,
        scoring=negAIC
    )
    print(f"✅ Stepwise para {aglo}")

    # Ajustamos el modelo
    modelo.fit(X_train, y_train)
    modelos[aglo] = modelo 
    print(f"✅ Modelo ajustado para {aglo}")

Trabajando con Resistencia...
✅ Stepwise para Resistencia
Optimization terminated successfully.
         Current function value: 0.577853
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.567472
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.577527
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.576206
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.571118
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.574597
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.573756
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.574796
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572928
         Iterations 5
Optimization terminated 

LinAlgError: Singular matrix

In [ ]:
design = ModelSpec(Hitters.columns.drop('Salary')).fit(Hitters)
Y = np.array(Hitters['Salary'])
X = design.transform(Hitters)

strategy = Stepwise.first_peak(design,
                               direction='forward',
                               max_terms=len(design.terms))

hitters_Cp = sklearn_selected(OLS,
                              strategy,
                              scoring=negAIC)
hitters_Cp.fit(Hitters, Y)
hitters_Cp.selected_state_

In [176]:
# Definimos el modelo base
def build_model():
    return make_pipeline(
        StandardScaler(),
        LogisticRegression(class_weight='balanced')
    )

# Diccionario con los datasets
datasets = {
    "Resistencia": (X_R_train, y_R_train),
    "Corrientes": (X_C_train, y_C_train),
    "Formosa": (X_F_train, y_F_train),
    "Posadas": (X_P_train, y_P_train)
}

# Entrenar un modelo SFS por cada dataset
modelos_sfs = {}
for nombre, (X_train, y_train) in datasets.items():
    modelo = build_model()
    sfs = SequentialFeatureSelector(modelo, direction='forward')
    sfs.fit(X_train, y_train)
    modelos_sfs[nombre] = sfs
    print(f"✅ Modelo entrenado para {nombre}")

✅ Modelo entrenado para Resistencia
✅ Modelo entrenado para Corrientes



KeyboardInterrupt



In [154]:
modelos_sfs['Posadas'].get_support().shape
individual_NEA.shape

(15810, 89)

In [158]:
X_R_train.columns[modelos_sfs['Resistencia'].get_support()]

Index(['otros_ing_nolab', 'casadounido', 'IV5', 'IV6', 'IV7', 'IV8', 'IV9',
       'IV10', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II3_1', 'II6_1', 'II8',
       'II9', 'cantidad_ocupados', 'cantidad_desocupados',
       'NIVEL_ED_Secundario incompleto', 'CALIFICACION_No calificados',
       'CALIFICACION_Profesionales', 'CALIFICACION_Técnicos',
       'JERARQUIA_Dirección', 'JERARQUIA_Jefes', 'JERARQUIA_Ns.Nc',
       'caes_seccion_cod_B', 'caes_seccion_cod_C', 'caes_seccion_cod_D',
       'caes_seccion_cod_E', 'caes_seccion_cod_F', 'caes_seccion_cod_H',
       'caes_seccion_cod_I', 'caes_seccion_cod_J', 'caes_seccion_cod_L',
       'caes_seccion_cod_M', 'caes_seccion_cod_N', 'caes_seccion_cod_P',
       'caes_seccion_cod_Q', 'caes_seccion_cod_R', 'caes_seccion_cod_S',
       'caes_seccion_cod_T', 'caes_seccion_cod_Z', 'PP04B1_Presta servicio'],
      dtype='object')

In [159]:
# Función para calcular métricas
def calcular_metricas(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    y_prob = modelo.predict_proba(X_test)[:,1] if hasattr(modelo, "predict_proba") else None
    
    acc = accuracy_score(y_test, y_pred)
    sens = recall_score(y_test, y_pred, pos_label=1)   # Sensibilidad
    esp = recall_score(y_test, y_pred, pos_label=0)   # Especificidad
    vpp = precision_score(y_test, y_pred, pos_label=1) # Valor Predictivo Positivo
    vpn = precision_score(y_test, y_pred, pos_label=0) # Valor Predictivo Negativo
    
    auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
    
    return {
        "accuracy": round(acc,5),
        "sensibilidad": round(sens,5),
        "especificidad": round(esp,5),
        "VPP": round(vpp,5),
        "VPN": round(vpn,5),
        "auc": round(auc,5) if auc is not None else None
    }

# Diccionario con datasets de train/test
datasets = {
    "Resistencia": (X_R_train, X_R_test, y_R_train, y_R_test),
    "Corrientes": (X_C_train, X_C_test, y_C_train, y_C_test),
    "Formosa": (X_F_train, X_F_test, y_F_train, y_F_test),
    "Posadas": (X_P_train, X_P_test, y_P_train, y_P_test)
}

# DataFrame vacío para guardar resultados
resultados = pd.DataFrame()

In [174]:
resistencia = list(datasets.items())[0:2]
len(resistencia)

2

In [175]:
# Entrenamos y evaluamos
for aglo, (X_train, X_test, y_train, y_test) in resistencia:
    modelo = modelos_sfs[nombre]
    modelo.fit(X_train, y_train)
    print(f"✅ Modelo entrenado para {aglo}")
    
    metricas = calcular_metricas(modelo, X_test, y_test)
    metricas["Modelo"] = "Logistic Regression"
    metricas["Aglomerado"] = aglo
    print(f"✅ Métricas calculadas para {aglo}")
    
    resultados = pd.concat([resultados, pd.DataFrame([metricas])], ignore_index=True)
    print(f"✅ Resultados guardados para {nombre}")

print(resultados)

✅ Modelo entrenado para Resistencia


AttributeError: 'SequentialFeatureSelector' object has no attribute 'predict'

In [92]:
print(sfs.get_support())
individual_NEA_train.columns[sfs.get_support()]

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False]


Index(['completo', 'nro_rep', 'periodo', 'PONDERA_repr', 'viviendasxarea',
       'IPCF_d', 'otros_ing_nolab', 'RDECOCUR', 'ADECOCUR', 'CH06', 'CH06_2',
       'IX_TOT', 'informal', 'mujer', 'casadounido', 'ESTADO', 'CAT_INAC',
       'CAT_OCUP', 'IV1', 'IV2', 'IV3', 'IV4', 'IV5', 'IV6', 'IV7', 'IV8',
       'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II2',
       'II3', 'II3_1', 'II4_1', 'II4_2', 'II4_3', 'II5', 'II6', 'II6_1', 'II7',
       'II8', 'II9'],
      dtype='object')

Agregar que el estratificado incluya periodo y aglomerado también.